# PANDAS ALTERNATIVES IN PYTHON

- System: 

    Mackbook Pro 15 Inch, 2019 
        
        Processor: 2.3 GHz 8-Core Intel Core i9
        
        Memory: 16 GB 2400 MHz DDR4
        
        macOS:  Sonoma 14.5 Beta (23F5049f)

- Python 3.10.9

______


In [1]:
# Importing libraries 
import polars as pl
import datatable  as dt
import os
os.environ['MODIN_ENGINE'] = 'dask'
import modin.pandas as md
import modin
print("NUM Partitions available: ", modin.config.NPartitions.get())
# os.environ["MODIN_CPUS"] = "4"
import pandas as pd
import time
import numpy as np
from statistics import mean, stdev
import vaex as vx

NUM Partitions available:  16


In [2]:
def exec_time(code_str):
    start = time.time()
    exec(code_str)
    end = time.time()
    return np.round((end-start)/60*1000, 6)

def functiontiming(cmd_d, metr_nm, dictionary, loop = 10, add_cmd = None):
    print(metr_nm)
    for key, cmd in cmd_d.items():
        counter = 0
        times = []
        try: 
            while counter <= loop:
                if add_cmd:
                    new_cmd = add_cmd[key]
                    exec_time(new_cmd)
                time = exec_time(cmd)
                times.append(time)
                counter +=1
            meant = np.round(mean(times), 6)
            stdevt = np.round(stdev(times), 6)
            maxt = np.round(max(times), 6)
            mint = np.round(min(times), 6)
            add_dic = {'METRIC': metr_nm, 'LIBRARY': key, 'TIME (avg)': meant, "TIME (stdv)": stdevt, "TIME (max)": maxt, "TIME (min)": mint,   'N': loop}

        except Exception as E:
            print(key, " ERROR:\n", str(E))
            add_dic = {}

        if metr_nm in dictionary:
            dictionary[metr_nm].append(add_dic)
        else: 
            dictionary[metr_nm] = [add_dic]
        try: 
            print(key, '\t', meant, 'usec')     
        except: print('No data')
        
def dict_to_df(dictionary, file_size = None):
    df = pd.DataFrame()
    for k, v in dictionary.items(): 
        df1 = pd.DataFrame(v)
        df = pd.concat([df, df1], axis = 0).reset_index(drop = True)
    if file_size: 
        df.loc[:,'FILE_SZ'] = file_size
    return df  

______ 
### ~SMALL FILE 

In [3]:
# Number of data points to calculate statictics 
loops = 5
file = 'data/data_small.csv'
print("File size: ", np.round(os.stat(file).st_size / (1024 * 1024), 2), "MB")
RESULT_SM = {}

File size:  13.84 MB


In [4]:
cmdsrd = {
        'datatable': 'global dtdf ; dtdf = dt.fread(file)',
        'pandas'   : 'global pdf  ; pdf  = pd.read_csv(file, low_memory = False )',
        'polars'   : 'global pldf ; pldf = pl.read_csv(file, infer_schema_length=100000, ignore_errors = True )',
        'modin'    : 'global mdf  ; mdf  = md.read_csv(file, low_memory = False)',
        'vaex'     : 'global vxdf ; vxdf = vx.open(file)'
        }
metric = 'READ_CSV'
functiontiming(cmdsrd, metric, RESULT_SM, loop = loops)

READ_CSV
datatable 	 2.180515 usec
pandas 	 9.385989 usec
polars 	 7.471109 usec


To remove this warning, run the following python code before doing dataframe operations:

    from distributed import Client

    client = Client()

Dask needs bokeh >= 2.4.2, < 3 for the dashboard.
You have bokeh==3.0.3.
Continuing without the dashboard.
2024-04-28 19:26:24,947 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/dg/fckc2gz96c599j8pqfzz6jzr0000gn/T/dask-worker-space/worker-4fwssqu4', purging
2024-04-28 19:26:24,949 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/dg/fckc2gz96c599j8pqfzz6jzr0000gn/T/dask-worker-space/worker-x33fjr_k', purging
2024-04-28 19:26:24,951 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/dg/fckc2gz96c599j8pqfzz6jzr0000gn/T/dask-worker-space/worker-zihxvxo4', purging
2024-04-28 19:26:24,952 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/dg/fckc2gz96c599j8pqfzz6jzr0000gn/T/dask-worker-space/worker-5iqqcqqg', purg

modin 	 29.695955 usec
vaex 	 6.660628 usec


In [5]:
cmdshp = {
        'datatable': 'dtdf.shape',
        'pandas'   : 'pdf.shape',
        'polars'   : 'pldf.shape',
        'modin'    : 'mdf.shape',
        'vaex'     : 'vxdf.shape'
        }
metric = 'PRINT DF SHAPE'

functiontiming(cmdshp, metric, RESULT_SM, loop = loops)

PRINT DF SHAPE
datatable 	 0.000405 usec
pandas 	 0.000426 usec
polars 	 0.000504 usec
modin 	 0.001007 usec
vaex 	 0.001059 usec


In [6]:
cmds_copy = {
         'datatable': 'global dtdf1 ; dtdf1 = dtdf.copy()',
         'pandas'  : 'global pdf1  ; pdf1 = pdf.copy()',
         'polars'  : 'global pldf1 ; pldf1 = pldf.clone()',
         'modin'   : 'global mdf1  ; mdf1 = mdf.copy()',
         'vaex'    : 'global vxdf1 ; vxdf1 = vxdf.copy()'
        }
metric = 'CREATE COPY'
functiontiming(cmds_copy, metric, RESULT_SM, loop = loops)

CREATE COPY
datatable 	 0.000735 usec
pandas 	 0.330167 usec
polars 	 0.000908 usec
modin 	 0.008717 usec
vaex 	 0.065888 usec


In [7]:
cmds_col1 = {
         'datatable': 'global dtdf1 ; dtdf1.names = {"CRASH_CRN":"CRASH_CRNnew"}',
         'pandas'   : 'global pdf1  ; pdf1 = pdf1.rename(columns = {"CRASH_CRN":"CRASH_CRNnew"})',
         'polars'   : 'global pldf1 ; pldf1 = pldf1.rename({"CRASH_CRN":"CRASH_CRNnew"})',
         'modin'    : 'global mdf1  ; mdf1 = mdf1.rename(columns = {"CRASH_CRN":"CRASH_CRNnew"})',
         'vaex'     :  'vxdf1.rename("CRASH_CRN","CRASH_CRNnew")'
        }
metric = 'RENAME SINGLE COLUMN'
functiontiming(cmds_col1, metric, RESULT_SM, add_cmd = cmds_copy, loop = loops)

RENAME SINGLE COLUMN
datatable 	 0.00133 usec
pandas 	 0.228898 usec
polars 	 0.003272 usec
modin 	 0.059962 usec
vaex 	 0.067348 usec


In [8]:
new_columns = [col+'NEW' for col in pdf.columns]
new_colums_dict = {}
for col in pdf.columns:
    new_colums_dict[col] = col+'NEW'

cmds_col_all = {
         'datatable': 'global dtdf  ; dtdf.names = new_columns',
         'pandas'   : 'global pdf   ; pdf.columns = new_columns',
         # For polars to work with no errors I had to create a new dataframe. 
         # Tests without new copy in other platforms worked with no issues
         'polars'   : 'global pldf2 ; pldf2 =  pldf.rename(new_colums_dict)',
         'modin'    : 'global mdf   ; mdf = mdf.rename(columns = new_colums_dict)',
         'vaex'     : 'for cur_nm, new_nm in new_colums_dict.items(): vxdf1.rename(cur_nm, new_nm)'
        }
metric = 'RENAME ALL COLUMNS'
functiontiming(cmds_col_all, metric, RESULT_SM, add_cmd = cmds_copy, loop = loops)

RENAME ALL COLUMNS
datatable 	 0.001216 usec
pandas 	 0.0059 usec
polars 	 0.023639 usec
modin 	 0.059767 usec
vaex 	 4.59299 usec


In [9]:
cmds_sort1 = {
         'datatable': 'dtdf[:,:, dt.sort("MUNICIPALITYNEW", reverse=True)]',
         'pandas'   : 'pdf.sort_values(by = ["MUNICIPALITYNEW"], ascending = [False])',
         'polars'   : 'pldf2.sort("MUNICIPALITYNEW", descending=True)',
         'modin'    : 'mdf.sort_values(by = ["MUNICIPALITYNEW"], ascending = [False])',
         'vaex'     : 'vxdf.sort(["MUNICIPALITY"])'
        }
metric = 'SORT ONE COLUMN'
functiontiming(cmds_sort1, metric, RESULT_SM, loop = loops)

SORT ONE COLUMN
datatable 	 0.018063 usec
pandas 	 0.392808 usec
polars 	 0.388208 usec
modin 	 143.579346 usec
vaex 	 2.288666 usec


In [10]:
cmds_sort2 = {
         'datatable': 'dtdf[:,:, dt.sort(["MUNICIPALITYNEW", "CRASH_YEARNEW"], reverse=[True, False])]',
         'pandas'   : 'pdf.sort_values(by = ["MUNICIPALITYNEW", "CRASH_YEARNEW"], ascending = [False, True])',
         'polars'   : 'pldf2.sort("MUNICIPALITYNEW", "CRASH_YEARNEW", descending=[True, False])',
         'modin'    : 'mdf.sort_values(by = ["MUNICIPALITYNEW", "CRASH_YEARNEW"], ascending = [False, True])',
         'vaex'     : 'vxdf.sort(["MUNICIPALITY", "CRASH_YEAR"], ascending = [False, True])'
        }
metric = 'SORT TWO COLUMN'
functiontiming(cmds_sort2, metric, RESULT_SM, loop = loops)

SORT TWO COLUMN
datatable 	 0.086756 usec
pandas 	 0.41567 usec
polars 	 0.302688 usec
modin 	 139.256868 usec
vaex 	 3.707558 usec


In [11]:
from datatable import dt, f, by
grp_by_sum = {
         'datatable': 'dtdf[:, dt.sum(f.CRASH_YEARNEW), by("MUNICIPALITYNEW")]',
         'pandas'   : 'pdf.groupby("MUNICIPALITYNEW")["CRASH_YEARNEW"].sum()',
         'polars'   : 'pldf2.group_by("MUNICIPALITYNEW").agg(pl.sum("CRASH_YEARNEW"))',
         'modin'    : 'mdf.groupby("MUNICIPALITYNEW")["CRASH_YEARNEW"].sum()',
         'vaex'     : "vxdf.groupby(by='MUNICIPALITY').agg({'CRASH_YEAR': 'sum'})"

        }
metric = 'GROUP BY SUM'
functiontiming(grp_by_sum, metric, RESULT_SM, loop = loops)

GROUP BY SUM
datatable 	 0.018153 usec
pandas 	 0.028611 usec
polars 	 0.182442 usec
modin 	 12.965337 usec
vaex 	 10.608231 usec


In [12]:
dict_to_df(RESULT_SM, 'Small')

,METRIC,LIBRARY,TIME (avg),TIME (stdv),TIME (max),TIME (min),N,FILE_SZ
0,READ_CSV,datatable,2.180515,0.286815,2.760903,2.001536,5,Small
1,READ_CSV,pandas,9.385989,0.491436,9.973049,8.700069,5,Small
2,READ_CSV,polars,7.471109,0.389008,8.078452,7.112149,5,Small
3,READ_CSV,modin,29.695955,44.752793,121.036216,10.645151,5,Small
4,READ_CSV,vaex,6.660628,6.623980,20.163584,3.642901,5,Small
5,PRINT DF SHAPE,datatable,0.000405,0.000143,0.000683,0.000318,5,Small
6,PRINT DF SHAPE,pandas,0.000426,0.000155,0.000735,0.000334,5,Small
7,PRINT DF SHAPE,polars,0.000504,0.000367,0.001248,0.000314,5,Small
8,PRINT DF SHAPE,modin,0.001007,0.000534,0.002082,0.000656,5,Small
9,PRINT DF SHAPE,vaex,0.001059,0.000238,0.001534,0.000918,5,Small


_____

### ~Medium file

In [13]:
### Deleting dataframes used with the Small file data
del dtdf, dtdf1, pdf, pdf1, pldf, pldf1, pldf2, mdf, mdf1, vxdf, vxdf1

In [14]:
file = 'data/data_medium.csv'
print("File size: ", np.round(os.stat(file).st_size / (1024 * 1024), 2), "MB")
RESULT_MD = {}

File size:  140.68 MB


In [15]:
metric = 'READ_CSV'
functiontiming(cmdsrd, metric, RESULT_MD, loop = loops)
metric = 'PRINT DF SHAPE'
functiontiming(cmdshp, metric, RESULT_MD, loop = loops)
metric = 'CREATE COPY'
functiontiming(cmds_copy, metric, RESULT_MD, loop = loops)
metric = 'RENAME SINGLE COLUMN'
functiontiming(cmds_col1, metric, RESULT_MD, add_cmd = cmds_copy, loop = loops)
metric = 'RENAME ALL COLUMNS'
functiontiming(cmds_col_all, metric, RESULT_MD, add_cmd = cmds_copy, loop = loops)
metric = 'SORT ONE COLUMN'
functiontiming(cmds_sort1, metric, RESULT_MD, loop = loops)
metric = 'SORT TWO COLUMN'
functiontiming(cmds_sort2, metric, RESULT_MD, loop = loops)
metric = 'GROUP BY SUM'
functiontiming(grp_by_sum, metric, RESULT_MD, loop = loops)

READ_CSV
datatable 	 8.880065 usec
pandas 	 135.542074 usec
polars 	 38.003842 usec
modin 	 38.602602 usec
vaex 	 7.470361 usec
PRINT DF SHAPE
datatable 	 0.000493 usec
pandas 	 0.000434 usec
polars 	 0.000783 usec
modin 	 0.001158 usec
vaex 	 0.003631 usec
CREATE COPY
datatable 	 0.000892 usec
pandas 	 6.948038 usec
polars 	 0.001204 usec
modin 	 0.008076 usec
vaex 	 0.13136 usec
RENAME SINGLE COLUMN
datatable 	 0.00149 usec
pandas 	 6.244391 usec
polars 	 0.002592 usec
modin 	 0.077611 usec
vaex 	 0.104641 usec
RENAME ALL COLUMNS
datatable 	 0.003886 usec
pandas 	 0.007077 usec
polars 	 0.016958 usec
modin 	 0.061813 usec
vaex 	 4.939165 usec
SORT ONE COLUMN
datatable 	 0.026308 usec
pandas 	 6.193146 usec
polars 	 4.216641 usec
modin 	 170.131441 usec
vaex 	 12.942906 usec
SORT TWO COLUMN
datatable 	 0.255921 usec
pandas 	 8.962722 usec
polars 	 3.347264 usec


2024-04-28 19:32:05,706 - distributed.worker - WARNING - Could not find data: {'function-5d575a7d450a477b95e867b63d7322b3': ['tcp://127.0.0.1:54607', 'tcp://127.0.0.1:54616', 'tcp://127.0.0.1:54599', 'tcp://127.0.0.1:54628', 'tcp://127.0.0.1:54637', 'tcp://127.0.0.1:54615', 'tcp://127.0.0.1:54622', 'tcp://127.0.0.1:54598', 'tcp://127.0.0.1:54619']} on workers: [] (who_has: {'function-5d575a7d450a477b95e867b63d7322b3': ['tcp://127.0.0.1:54607', 'tcp://127.0.0.1:54616', 'tcp://127.0.0.1:54599', 'tcp://127.0.0.1:54628', 'tcp://127.0.0.1:54637', 'tcp://127.0.0.1:54615', 'tcp://127.0.0.1:54622', 'tcp://127.0.0.1:54598', 'tcp://127.0.0.1:54619']})
2024-04-28 19:32:05,707 - distributed.scheduler - WARNING - Worker tcp://127.0.0.1:54634 failed to acquire keys: {'function-5d575a7d450a477b95e867b63d7322b3': ('tcp://127.0.0.1:54607', 'tcp://127.0.0.1:54616', 'tcp://127.0.0.1:54599', 'tcp://127.0.0.1:54628', 'tcp://127.0.0.1:54637', 'tcp://127.0.0.1:54615', 'tcp://127.0.0.1:54622', 'tcp://127.0.0.

modin 	 157.080745 usec
vaex 	 23.586046 usec
GROUP BY SUM
datatable 	 0.086062 usec
pandas 	 0.08951 usec
polars 	 1.120465 usec
modin 	 13.04252 usec
vaex 	 28.051453 usec


In [16]:
dict_to_df(RESULT_MD, 'Medium')

,METRIC,LIBRARY,TIME (avg),TIME (stdv),TIME (max),TIME (min),N,FILE_SZ
0,READ_CSV,datatable,8.880065,2.711761,14.302631,7.257346,5,Medium
1,READ_CSV,pandas,135.542074,2.961541,140.175903,132.453398,5,Medium
2,READ_CSV,polars,38.003842,3.405680,43.921514,35.519850,5,Medium
3,READ_CSV,modin,38.602602,3.342747,44.505366,36.026851,5,Medium
4,READ_CSV,vaex,7.470361,2.268720,10.796352,5.246333,5,Medium
5,PRINT DF SHAPE,datatable,0.000493,0.000144,0.000679,0.000370,5,Medium
6,PRINT DF SHAPE,pandas,0.000434,0.000141,0.000715,0.000350,5,Medium
7,PRINT DF SHAPE,polars,0.000783,0.000289,0.001351,0.000548,5,Medium
8,PRINT DF SHAPE,modin,0.001158,0.000502,0.001915,0.000648,5,Medium
9,PRINT DF SHAPE,vaex,0.003631,0.002998,0.009203,0.001462,5,Medium


_____

### ~Large file

In [17]:
### Deleting dataframes used with the Medium file data
del dtdf, dtdf1, pdf, pdf1, pldf, pldf1, pldf2, mdf, mdf1, vxdf, vxdf1

In [18]:
file = 'data/data_large.csv'
print("File size: ", np.round(os.stat(file).st_size / (1024 * 1024), 2), "MB")
RESULT_LG = {}

File size:  281.36 MB


In [19]:
metric = 'READ_CSV'
functiontiming(cmdsrd, metric, RESULT_LG, loop = loops)
metric = 'PRINT DF SHAPE'
functiontiming(cmdshp, metric, RESULT_LG, loop = loops)
metric = 'CREATE COPY'
functiontiming(cmds_copy, metric, RESULT_LG, loop = loops)
metric = 'RENAME SINGLE COLUMN'
functiontiming(cmds_col1, metric, RESULT_LG, add_cmd = cmds_copy, loop = loops)
metric = 'RENAME ALL COLUMNS'
functiontiming(cmds_col_all, metric, RESULT_LG, add_cmd = cmds_copy, loop = loops)
metric = 'SORT ONE COLUMN'
functiontiming(cmds_sort1, metric, RESULT_LG, loop = loops)
metric = 'SORT TWO COLUMN'
functiontiming(cmds_sort2, metric, RESULT_LG, loop = loops)
metric = 'GROUP BY SUM'
functiontiming(grp_by_sum, metric, RESULT_LG, loop = loops)

READ_CSV
datatable 	 12.466177 usec
pandas 	 265.811959 usec
polars 	 41.484716 usec
modin 	 71.544194 usec
vaex 	 6.068185 usec
PRINT DF SHAPE
datatable 	 0.000508 usec
pandas 	 0.00043 usec
polars 	 0.000783 usec
modin 	 0.000995 usec
vaex 	 0.001198 usec
CREATE COPY
datatable 	 0.001066 usec
pandas 	 12.074617 usec
polars 	 0.001089 usec
modin 	 0.008029 usec
vaex 	 0.074164 usec
RENAME SINGLE COLUMN
datatable 	 0.001547 usec
pandas 	 10.514478 usec
polars 	 0.007502 usec
modin 	 0.06067 usec
vaex 	 0.068167 usec
RENAME ALL COLUMNS
datatable 	 0.013574 usec
pandas 	 0.005728 usec
polars 	 0.043584 usec
modin 	 0.061666 usec
vaex 	 4.948886 usec
SORT ONE COLUMN
datatable 	 0.050322 usec
pandas 	 16.755631 usec
polars 	 10.606325 usec


2024-04-28 19:35:41,282 - distributed.worker.memory - WARNING - Worker is at 83% memory usage. Pausing worker.  Process memory: 852.41 MiB -- Worker memory limit: 1.00 GiB
2024-04-28 19:35:41,642 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:54607 (pid=19194) exceeded 95% memory budget. Restarting...
2024-04-28 19:35:42,216 - distributed.nanny - WARNING - Restarting worker
2024-04-28 19:35:46,293 - distributed.worker.memory - WARNING - Worker is at 89% memory usage. Pausing worker.  Process memory: 913.29 MiB -- Worker memory limit: 1.00 GiB
2024-04-28 19:35:46,399 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:54604 (pid=19192) exceeded 95% memory budget. Restarting...
2024-04-28 19:35:46,526 - distributed.nanny - WARNING - Restarting worker
2024-04-28 19:35:49,767 - distributed.worker.memory - WARNING - Worker is at 83% memory usage. Pausing worker.  Process memory: 850.32 MiB -- Worker memory limit: 1.00 GiB
2024-04-28 19:35:49,875 - distributed.nanny.

modin  ERROR:
 lambda-d3950906e45b4b068f222cc5a451e9bf
modin 	 10.606325 usec


2024-04-28 19:36:00,089 - distributed.worker - ERROR - Exception during execution of task deploy_dask_func-d6a4be2f-7cd5-4e90-8d82-3bdb822eb43f.
Traceback (most recent call last):
  File "/Users/jorgepinzon/opt/anaconda3/envs/py310/lib/python3.10/site-packages/distributed/worker.py", line 2366, in _prepare_args_for_execution
    data[k] = self.data[k]
  File "/Users/jorgepinzon/opt/anaconda3/envs/py310/lib/python3.10/site-packages/distributed/spill.py", line 257, in __getitem__
    return super().__getitem__(key)
  File "/Users/jorgepinzon/opt/anaconda3/envs/py310/lib/python3.10/site-packages/zict/buffer.py", line 108, in __getitem__
    raise KeyError(key)
KeyError: 'lambda-6ab893161f39db276beb204c0b4b6fa2'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jorgepinzon/opt/anaconda3/envs/py310/lib/python3.10/site-packages/distributed/worker.py", line 2247, in execute
    args2, kwargs2 = self._prepare_args_for_execut

vaex 	 28.895834 usec
SORT TWO COLUMN
datatable 	 0.281839 usec
pandas 	 17.64547 usec
polars 	 12.881739 usec


2024-04-28 19:36:26,938 - distributed.worker.memory - WARNING - Worker is at 82% memory usage. Pausing worker.  Process memory: 843.52 MiB -- Worker memory limit: 1.00 GiB
2024-04-28 19:36:27,454 - distributed.worker.memory - WARNING - Worker is at 74% memory usage. Resuming worker. Process memory: 758.81 MiB -- Worker memory limit: 1.00 GiB
2024-04-28 19:36:27,785 - distributed.worker.memory - WARNING - Worker is at 82% memory usage. Pausing worker.  Process memory: 844.48 MiB -- Worker memory limit: 1.00 GiB
2024-04-28 19:36:28,447 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:55497 (pid=22221) exceeded 95% memory budget. Restarting...
2024-04-28 19:36:29,447 - distributed.nanny - WARNING - Restarting worker
2024-04-28 19:36:33,291 - distributed.worker.memory - WARNING - Worker is at 81% memory usage. Pausing worker.  Process memory: 834.68 MiB -- Worker memory limit: 1.00 GiB
2024-04-28 19:36:33,448 - distributed.worker.memory - WARNING - Worker is at 68% memory usag

modin  ERROR:
 Attempted to run task deploy_dask_func-f696886b-5420-48ac-8cf1-7ff0758f423b on 3 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:55623. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.
modin 	 12.881739 usec
vaex 	 52.840092 usec
GROUP BY SUM
datatable 	 0.158898 usec
pandas 	 0.212437 usec
polars 	 2.686306 usec
modin 	 14.727121 usec
vaex 	 53.103473 usec


In [20]:
dict_to_df(RESULT_LG, 'Large')

,METRIC,LIBRARY,TIME (avg),TIME (stdv),TIME (max),TIME (min),N,FILE_SZ
0,READ_CSV,datatable,12.466177,1.161104,14.387182,11.533419,5.0,Large
1,READ_CSV,pandas,265.811959,7.681462,275.931839,253.054734,5.0,Large
2,READ_CSV,polars,41.484716,1.910927,44.709917,39.967589,5.0,Large
3,READ_CSV,modin,71.544194,8.202934,85.593065,64.507099,5.0,Large
4,READ_CSV,vaex,6.068185,2.546072,11.226066,4.519550,5.0,Large
5,PRINT DF SHAPE,datatable,0.000508,0.000345,0.001204,0.000334,5.0,Large
6,PRINT DF SHAPE,pandas,0.000430,0.000133,0.000699,0.000350,5.0,Large
7,PRINT DF SHAPE,polars,0.000783,0.000995,0.002813,0.000330,5.0,Large
8,PRINT DF SHAPE,modin,0.000995,0.001130,0.003298,0.000481,5.0,Large
9,PRINT DF SHAPE,vaex,0.001198,0.000479,0.002170,0.000966,5.0,Large


____

# Combining the results

In [21]:
results = dict_to_df(RESULT_LG).drop(['N'], axis = 1).merge(dict_to_df(RESULT_MD), on = ['METRIC', 'LIBRARY'], suffixes=['_LG', '_MD']).drop(['N'], axis = 1).merge(dict_to_df(RESULT_SM), on = ['METRIC', 'LIBRARY'])
ordered_columns = ['METRIC', 'LIBRARY', 'TIME (avg)_LG', 'TIME (avg)_MD', 'TIME (avg)',
                    'TIME (stdv)_LG', 'TIME (stdv)_MD', 'TIME (stdv)', 
                    'TIME (max)_LG', 'TIME (max)_MD', 'TIME (max)',
                    'TIME (min)_LG', 'TIME (min)_MD', 'TIME (min)', 'N']
results[ordered_columns]

,METRIC,LIBRARY,TIME (avg)_LG,TIME (avg)_MD,TIME (avg),TIME (stdv)_LG,TIME (stdv)_MD,TIME (stdv),TIME (max)_LG,TIME (max)_MD,TIME (max),TIME (min)_LG,TIME (min)_MD,TIME (min),N
0,READ_CSV,datatable,12.466177,8.880065,2.180515,1.161104,2.711761,0.286815,14.387182,14.302631,2.760903,11.533419,7.257346,2.001536,5
1,READ_CSV,pandas,265.811959,135.542074,9.385989,7.681462,2.961541,0.491436,275.931839,140.175903,9.973049,253.054734,132.453398,8.700069,5
2,READ_CSV,polars,41.484716,38.003842,7.471109,1.910927,3.405680,0.389008,44.709917,43.921514,8.078452,39.967589,35.519850,7.112149,5
3,READ_CSV,modin,71.544194,38.602602,29.695955,8.202934,3.342747,44.752793,85.593065,44.505366,121.036216,64.507099,36.026851,10.645151,5
4,READ_CSV,vaex,6.068185,7.470361,6.660628,2.546072,2.268720,6.623980,11.226066,10.796352,20.163584,4.519550,5.246333,3.642901,5
5,PRINT DF SHAPE,datatable,0.000508,0.000493,0.000405,0.000345,0.000144,0.000143,0.001204,0.000679,0.000683,0.000334,0.000370,0.000318,5
6,PRINT DF SHAPE,pandas,0.000430,0.000434,0.000426,0.000133,0.000141,0.000155,0.000699,0.000715,0.000735,0.000350,0.000350,0.000334,5
7,PRINT DF SHAPE,polars,0.000783,0.000783,0.000504,0.000995,0.000289,0.000367,0.002813,0.001351,0.001248,0.000330,0.000548,0.000314,5
8,PRINT DF SHAPE,modin,0.000995,0.001158,0.001007,0.001130,0.000502,0.000534,0.003298,0.001915,0.002082,0.000481,0.000648,0.000656,5
9,PRINT DF SHAPE,vaex,0.001198,0.003631,0.001059,0.000479,0.002998,0.000238,0.002170,0.009203,0.001534,0.000966,0.001462,0.000918,5
